# Practical Uses of SQL For Making Business Decisions

The goal for this project is to come up with business decisions based on SQL queries. This project is based off the Chinook Database, with the following schema.

<img src = 'chinook-schema.svg' width=500>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [11]:
%%sql
SELECT name, type FROM sqlite_master
WHERE type IN ('table', 'view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The first thing I am going to want to do is get a look at what tables and views the database has. In addition to all the tables in the schema, there are two views:

1. customer_usa: Customers in the USA
2. customer_gt_90_dollars: Customers who have purchased more than 90 dollars in music